# Neighborhood Score

- foreclosures in last 3 years / # of residences
- police dispatches per address / population of CT
- complaints in last 3 years / pop of CT
- median income
- % white
- % black
- % other
- % under 16
- % over 65
- median property value
- % vacant lots


Need to calculate:
- % homeownership
- Number of negative actions taken / # of residences

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
census_tracts = gpd.read_file('../Data/processed/shapefiles/CT_Demo_CH.shp')
foreclosures = gpd.read_file('../Data/processed/shapefiles/FC_count.shp')
police_dispatch = gpd.read_file('../Data/processed/shapefiles/PD_count.shp')
complaints = gpd.read_file('../Data/processed/shapefiles/complaint_count.shp')
property_info = gpd.read_file('../Data/processed/shapefiles/featureSpace.shp')

In [3]:
census_tracts = census_tracts[['Median_Inc', '65_Over_Pe', 'Under_16_P', 'White_Perc', 'Black_Perc', \
                               'Other_Perc', 'DP0010001', 'GEO_ID', 'NAMELSAD10', 'geometry']]

### Calculate count of residences in each census tract

In [4]:
property_info = property_info[['parcel_add', 'land_tax', 'vacant', 'geometry']]
property_info.drop_duplicates(['parcel_add', 'land_tax', 'vacant'], inplace=True)

In [5]:
property_ct = gpd.sjoin(census_tracts, property_info, how='inner', op='intersects')
property_ct.drop_duplicates(['parcel_add', 'land_tax', 'vacant'], inplace=True)

In [6]:
property_ct_count = property_ct.groupby('NAMELSAD10').count()
property_ct_count = property_ct_count[['Median_Inc']]
property_ct_count.reset_index(inplace=True)
property_ct_count.columns = ['NAMELSAD10', 'Res_Count']

In [7]:
census_tracts = pd.merge(census_tracts, property_ct_count, how='inner')

### Calculate foreclosures

In [8]:
foreclosure_ct = gpd.sjoin(census_tracts, foreclosures, how='left', op='intersects')

In [9]:
foreclosure_count = foreclosure_ct.groupby('NAMELSAD10').sum()
foreclosure_count = foreclosure_count[['Count']]
foreclosure_count.reset_index(inplace=True)
featureSpace = pd.merge(census_tracts, foreclosure_count, how='inner')

In [10]:
featureSpace.columns = ['Median_Inc', '%_Over65', '%_Under16', '%_White', '%_Black', '%_Other', \
                        'Population', 'GEOID', 'NAMELSAD10', 'geometry', 'res_count', 'FC_count']
featureSpace['FC_count'] = featureSpace.FC_count/featureSpace.res_count

### Calculate police dispatches

In [11]:
police_dispatch_ct = gpd.sjoin(census_tracts, police_dispatch, how='left', op='intersects')

In [12]:
police_dispatch_count = police_dispatch_ct.groupby('NAMELSAD10').sum()
police_dispatch_count = police_dispatch_count[['PD_Count']]
police_dispatch_count.reset_index(inplace=True)
featureSpace = pd.merge(featureSpace, police_dispatch_count, how='inner')
featureSpace['PD_Count'] = featureSpace['PD_Count']/featureSpace.Population

### Calculate complaints

In [13]:
complaints.columns = ['Address', 'comp_count', 'geometry']
complaints_ct = gpd.sjoin(census_tracts, complaints, how='left', op='intersects')

In [14]:
complaints_count = complaints_ct.groupby('NAMELSAD10').sum()
complaints_count = complaints_count[['comp_count']]
complaints_count.reset_index(inplace=True)
featureSpace = pd.merge(featureSpace, complaints_count, how='inner')
featureSpace['comp_count'] = featureSpace['comp_count']/featureSpace.Population

### Calculate median land tax value

In [15]:
property_info_ct = gpd.sjoin(census_tracts, property_info, how='left', op='intersects')

In [16]:
property_info_count = property_info_ct.groupby('NAMELSAD10').median()
property_info_count = property_info_count[['land_tax']]
property_info_count.reset_index(inplace=True)
featureSpace = pd.merge(featureSpace, property_info_count, how='inner')

### Calculate vacant lots

In [17]:
property_info_count = property_info_ct.groupby('NAMELSAD10').sum()
property_info_count = property_info_count[['vacant']]
property_info_count.reset_index(inplace=True)
featureSpace = pd.merge(featureSpace, property_info_count, how='inner')
featureSpace['vacant'] = featureSpace['vacant']/featureSpace['res_count']

### Clean feature space and output file

In [18]:
featureSpace.rename(columns={'FC_count': 'FC_Count', 'comp_count': 'Comp_Count', 'land_tax': 'Median_Val', \
                             'vacant': '%_Vacant'}, inplace=True)
featureSpace.fillna(0, inplace=True)
featureSpace.head()

,Median_Inc,%_Over65,%_Under16,%_White,%_Black,%_Other,Population,GEOID,NAMELSAD10,geometry,res_count,FC_Count,PD_Count,Comp_Count,Median_Val,%_Vacant
0,35100,0.131016,0.173797,0.073864,0.904746,0.021390,2992,1600000US3916014,Census Tract 1261,(POLYGON ((-81.55600099999998 41.5452369999999...,26,0.230769,0.000000,0.000000,119150.0,0.153846
1,60143,0.156047,0.215865,0.136541,0.824447,0.039012,1538,1600000US3916014,Census Tract 1401,POLYGON ((-81.55588699999998 41.53788500000002...,658,0.493921,0.911573,0.102081,81200.0,0.075988
2,40938,0.130759,0.222466,0.043440,0.930671,0.025889,2279,1600000US3916014,Census Tract 1403.01,"POLYGON ((-81.556631 41.52714300000014, -81.55...",991,0.523713,1.045195,0.007898,65100.0,0.066599
3,41582,0.176920,0.195163,0.327111,0.627917,0.044972,2357,1600000US3916014,Census Tract 1403.02,"POLYGON ((-81.55001399999998 41.520512, -81.55...",1021,0.373164,1.202800,0.036487,79700.0,0.037218
4,55227,0.129930,0.207159,0.384819,0.555519,0.059662,3017,1600000US3916014,Census Tract 1404,POLYGON ((-81.54627900000003 41.52064300000001...,1113,0.390836,1.501160,0.076898,92050.0,0.038634


In [19]:
featureSpace.to_file('../Data/processed/shapefiles/featureSpace_CT.shp')